# IUM projekt - temat 2

## Treść zadania
> Są osoby, które wchodzą na naszą stronę i nie mogą się zdecydować, którym produktom przyjrzeć się nieco lepiej. Może dało by się im coś polecić?

### Obliczenia
Poniżej znajdują się obliczenia, na których oparte zostały wartości liczbowe z powyższego raportu. Wszystkie obliczenia dotyczą bazy rekordów wydarzeń sesji.

In [3]:
import numpy as np
import pandas as pd

data_raw_dir = 'data_raw/'
data_dir = 'data/'
sessions_file = 'sessions.jsonl'
sessions_raw_filepath = data_raw_dir + sessions_file
sessions_filepath = data_dir + sessions_file

In [ ]:
# Dane wydarzeń sesji
sessions_raw_data = pd.read_json(sessions_raw_filepath, lines=True)
sessions_raw_data

#### Ilość błędnych wierszy w bazie rekordów sesji

In [ ]:
row_num_pre_drop = len(sessions_raw_data.index)
print('Ilość wierszy przed usunięciem błędnych danych: ', row_num_pre_drop)

data_cleaned = pd.read_json(sessions_filepath, lines=True)
row_num_after_drop = len(data_cleaned.index)
print('Ilość wierszy po odzyskaniu i usunięciu błędnych danych: ', row_num_after_drop)

fraction_lost = (row_num_pre_drop - row_num_after_drop) / row_num_pre_drop
print('Usuniętych danych: {:.2%}'.format(fraction_lost))

#### Średnia ilość przejrzeń produktów dziennie
Wliczane są dane uszkodzone, tzn. również te, w których brakuje części danych.

In [ ]:
# Ilość wyświetleń na dzień
view_event_timestamps = sessions_raw_data[sessions_raw_data['event_type']=='VIEW_PRODUCT']['timestamp']
view_events_per_day = (pd.to_datetime(view_event_timestamps)
    .dt.floor('d')
    .value_counts()
    .rename_axis('day')
    .reset_index(name='count')
    .sort_values('day'))
view_events_per_day

In [ ]:
# Średnia ilość wyświetleń na dzień
mean_view_events_per_day = view_events_per_day['count'][:-1].mean()
round(mean_view_events_per_day)

Stąd wynika, że wzrost wyświetleń o 10% to około 115 wyświetleń (zaokrąglając w górę).

#### Gęstość macierzy sąsiedztwa (adjacency matrix)

In [4]:
# Załadowanie wyczyszczonych danych sesji
sessions_data = pd.read_json(sessions_filepath, lines=True)
# Podział rekordów na rekordy dotyczące VIEW oraz BUY
views_data = sessions_data[sessions_data['event_type'].str.contains('VIEW_PRODUCT')]
buy_data = sessions_data[sessions_data['event_type'].str.contains('BUY_PRODUCT')]

In [5]:
# Macierze sąsiedztwa (z uwzględnieniem liczby przejrzeń i zakupów)
view_adjacency_matrix = pd.crosstab(views_data['user_id'], views_data['product_id'])
buy_adjacency_matrix = pd.crosstab(buy_data['user_id'], buy_data['product_id'])

In [8]:
def get_uniques(df, column_name) -> set:
    uniques = df[column_name].unique()
    uniques.sort()
    uniques = set(uniques)
    return uniques
    
# Ilości unikalnych użytkowników i produktów w danych sesji
items_in_sessions = get_uniques(sessions_data, 'product_id')
users_in_sessions = get_uniques(sessions_data, 'user_id')

In [10]:
total_possible_links = len(items_in_sessions)*len(users_in_sessions)

# Obliczanie gęstości VIEW
links_in_view_adjacency_matrix = view_adjacency_matrix.astype(bool).sum(axis=0).sum(axis=0)
view_density = links_in_view_adjacency_matrix / total_possible_links
# Obliczanie gęstości BUY
links_in_buy_adjacency_matrix = buy_adjacency_matrix.astype(bool).sum(axis=0).sum(axis=0)
buy_density = links_in_buy_adjacency_matrix / total_possible_links

print("View density: {:.3%}".format(view_density))
print("Buy density: {:.3%}".format(buy_density))

View density: 75.629%
Buy density: 8.638%
